M Kronig (23029039)

CSC-40098 Final Project

Analysis of posts from the CS Educators Stack Exchange website
(SS3) What questions are computer science eduators asking?

Installing packages needed for the project

In [2]:
# Importing packages needed for project
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import string
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords

Reading the data into a Pandas dataframe

In [3]:
# Reading the data into a Pandas dataframe
df = pd.read_csv('/Users/matthewkronig/Git/CSC-40098/CSED.csv')

Looking at the shape of the data, there are 6499 rows and 11 columns

In [4]:
# Looking a the shape of the data 
df.shape

(6499, 11)

Using the dataframe head() function to inspect the columns and a small set of the rows

In [5]:
df.head()

,Id,PostTypeId,ParentId,Title,Body,CommentCount,ViewCount,AnswerCount,Score,Tags,CreationDate
0,2,1,NaN,What autograding software do folks use for Jav...,<p>Grading currently either takes me a huge am...,9,5402.0,8.0,26,<java><grading><auto-graders>,2017-05-23 14:20:02
1,3,1,NaN,Is it possible to ensure division of labor on ...,<p>Let's say I've assigned my Intro to CS clas...,1,602.0,4.0,10,<group-work>,2017-05-23 14:20:35
2,4,1,NaN,Objects Lab Before Inheritance and Polymorphism,<p>I am finding it very difficult to create a ...,3,712.0,11.0,9,<object-oriented-paradigm><labs><ap-computer-s...,2017-05-23 14:21:14
3,6,1,NaN,Teaching Induction to Prove the Correctness of...,<p>This is a subject that I have had a lot of ...,1,413.0,3.0,8,<lesson-ideas><cs-theory><induction>,2017-05-23 14:24:14
4,7,1,NaN,Should the first Programming/Algorithms class ...,"<p>When I was studying, my professors had diff...",7,1404.0,9.0,22,<pseudo-code>,2017-05-23 14:25:23


Using the dtypes method to have a look a the the datatypes Pandas has assigned

In [6]:
df.dtypes

Id                int64
PostTypeId        int64
ParentId        float64
Title            object
Body             object
CommentCount      int64
ViewCount       float64
AnswerCount     float64
Score             int64
Tags             object
CreationDate     object
dtype: object

In [7]:
df.describe()

,Id,PostTypeId,ParentId,CommentCount,ViewCount,AnswerCount,Score
count,6499.000000,6499.000000,4786.000000,6499.00000,1250.000000,1250.000000,6499.000000
mean,4284.675181,1.986152,4015.998537,2.14156,1557.989600,3.828800,4.986459
std,2401.090579,0.811273,2454.415062,3.29656,4945.944223,3.948565,10.054123
min,2.000000,1.000000,2.000000,0.00000,7.000000,0.000000,-5.000000
25%,2689.500000,2.000000,1408.000000,0.00000,138.000000,1.000000,1.000000
50%,4437.000000,2.000000,4211.000000,1.00000,266.000000,3.000000,3.000000
75%,6321.500000,2.000000,6005.000000,3.00000,743.000000,5.000000,5.000000
max,8237.000000,7.000000,8227.000000,42.00000,108964.000000,31.000000,229.000000


Data Cleaning

1. Check for duplicates
2. Check for missing values
3. Remove time from creation date column & convert datatype to date
4. Standardise text for analysis

No rows are duplicated

In [8]:
df.loc[df.duplicated()]

,Id,PostTypeId,ParentId,Title,Body,CommentCount,ViewCount,AnswerCount,Score,Tags,CreationDate


Checking for missing values

In [9]:
df.isna().sum()

Id                 0
PostTypeId         0
ParentId        1713
Title           5249
Body              52
CommentCount       0
ViewCount       5249
AnswerCount     5249
Score              0
Tags            5249
CreationDate       0
dtype: int64

There's some metadata that I don't need for now. So I need to clean the data to only inlcude the questions and answers, while keepig the relevant columns.

In [10]:
df['PostTypeId'].value_counts()

PostTypeId
2    4786
1    1250
5     231
4     231
7       1
Name: count, dtype: int64

Making a new dataframe qa_df (questions and answers dataframe)

In [11]:
# Remember to work with this new dataframe, not original df
qa_df = df[df['PostTypeId'].isin([1, 2])].copy()

In [12]:
qa_df['PostTypeId'].value_counts()

PostTypeId
2    4786
1    1250
Name: count, dtype: int64

The data numbers for missing values makes sense and matches the counts for PostTypeId 1 (Questions) and 2 (Answers). Questions will not will not have a parent ID
and answers will not have titles, tags, view counts and answer counts.

In [13]:
qa_df.isna().sum()

Id                 0
PostTypeId         0
ParentId        1250
Title           4786
Body               0
CommentCount       0
ViewCount       4786
AnswerCount     4786
Score              0
Tags            4786
CreationDate       0
dtype: int64

In [14]:
# New dataframe shape with removed rows
qa_df.shape

(6036, 11)

Next, convert CreationDate to date datatype as it's currently an object (string). Also, remove time as it's not needed.

In [15]:
#qa_df['CreationDate'] = pd.to_datetime(qa_df['CreationDate']) - This still kept the time which may clutter up the charts
qa_df['CreationDate'] = pd.to_datetime(qa_df['CreationDate']).dt.normalize()

In [16]:
qa_df.dtypes

Id                       int64
PostTypeId               int64
ParentId               float64
Title                   object
Body                    object
CommentCount             int64
ViewCount              float64
AnswerCount            float64
Score                    int64
Tags                    object
CreationDate    datetime64[ns]
dtype: object

In [17]:
# Checking the dataframe again
qa_df.head()

,Id,PostTypeId,ParentId,Title,Body,CommentCount,ViewCount,AnswerCount,Score,Tags,CreationDate
0,2,1,NaN,What autograding software do folks use for Jav...,<p>Grading currently either takes me a huge am...,9,5402.0,8.0,26,<java><grading><auto-graders>,2017-05-23
1,3,1,NaN,Is it possible to ensure division of labor on ...,<p>Let's say I've assigned my Intro to CS clas...,1,602.0,4.0,10,<group-work>,2017-05-23
2,4,1,NaN,Objects Lab Before Inheritance and Polymorphism,<p>I am finding it very difficult to create a ...,3,712.0,11.0,9,<object-oriented-paradigm><labs><ap-computer-s...,2017-05-23
3,6,1,NaN,Teaching Induction to Prove the Correctness of...,<p>This is a subject that I have had a lot of ...,1,413.0,3.0,8,<lesson-ideas><cs-theory><induction>,2017-05-23
4,7,1,NaN,Should the first Programming/Algorithms class ...,"<p>When I was studying, my professors had diff...",7,1404.0,9.0,22,<pseudo-code>,2017-05-23


Next I need to start working on the format of the string objects - currently formatted as HTML, as well as finding a sensible way of organising the Tags data. The Title data isn't stored in HTML so I'll check this as well.


1. Lowercase everything

2. Strip HTML

3. Remove punctuation?

4. Remove stop words ("the", "is", "and") etc

5. Tokenization (splitting text into words)

6. Lemmatization or stemming — reducing words to their base/root form

7. Optional: Spell-check or correction, if the data is messy or shows errors

Removing HTML with function using BeautifulSoup

In [18]:

def clean_html(text):
     """
        Removes the HTML from the Posts to clear away unescessary tags for analysis using BeautifulSoup.

        Parameters:
        text (str): The HTML string to clean (e.g., post body or title).

        Returns:
        str: The cleaned text with HTML tags removed.
        If the input is NaN or not a valid string, it returns an empty string.
        """
     if pd.isna(text):
         return ""
     return BeautifulSoup(text, 'html.parser').get_text()


In [19]:
qa_df['Title'] = qa_df['Title'].apply(clean_html)
qa_df['Body'] = qa_df['Body'].apply(clean_html)

In [20]:
qa_df.head()

,Id,PostTypeId,ParentId,Title,Body,CommentCount,ViewCount,AnswerCount,Score,Tags,CreationDate
0,2,1,NaN,What autograding software do folks use for Jav...,Grading currently either takes me a huge amoun...,9,5402.0,8.0,26,<java><grading><auto-graders>,2017-05-23
1,3,1,NaN,Is it possible to ensure division of labor on ...,Let's say I've assigned my Intro to CS class t...,1,602.0,4.0,10,<group-work>,2017-05-23
2,4,1,NaN,Objects Lab Before Inheritance and Polymorphism,I am finding it very difficult to create a lab...,3,712.0,11.0,9,<object-oriented-paradigm><labs><ap-computer-s...,2017-05-23
3,6,1,NaN,Teaching Induction to Prove the Correctness of...,This is a subject that I have had a lot of tro...,1,413.0,3.0,8,<lesson-ideas><cs-theory><induction>,2017-05-23
4,7,1,NaN,Should the first Programming/Algorithms class ...,"When I was studying, my professors had differe...",7,1404.0,9.0,22,<pseudo-code>,2017-05-23


Making the Title and Body columns all lowercase

In [21]:
qa_df['Title'] = qa_df['Title'].str.lower() # Makes Title lowercase
qa_df['Body'] = qa_df['Body'].str.lower() # Makes Body lowercase

In [22]:
qa_df.head()

,Id,PostTypeId,ParentId,Title,Body,CommentCount,ViewCount,AnswerCount,Score,Tags,CreationDate
0,2,1,NaN,what autograding software do folks use for jav...,grading currently either takes me a huge amoun...,9,5402.0,8.0,26,<java><grading><auto-graders>,2017-05-23
1,3,1,NaN,is it possible to ensure division of labor on ...,let's say i've assigned my intro to cs class t...,1,602.0,4.0,10,<group-work>,2017-05-23
2,4,1,NaN,objects lab before inheritance and polymorphism,i am finding it very difficult to create a lab...,3,712.0,11.0,9,<object-oriented-paradigm><labs><ap-computer-s...,2017-05-23
3,6,1,NaN,teaching induction to prove the correctness of...,this is a subject that i have had a lot of tro...,1,413.0,3.0,8,<lesson-ideas><cs-theory><induction>,2017-05-23
4,7,1,NaN,should the first programming/algorithms class ...,"when i was studying, my professors had differe...",7,1404.0,9.0,22,<pseudo-code>,2017-05-23


Next, remove all punctuation from Title and Body columns

In [23]:
# Remove punctuation from both 'Title' and 'Body' columns
qa_df['Title'] = qa_df['Title'].str.translate(str.maketrans('', '', string.punctuation))
qa_df['Body'] = qa_df['Body'].str.translate(str.maketrans('', '', string.punctuation))


In [24]:
qa_df.head()

,Id,PostTypeId,ParentId,Title,Body,CommentCount,ViewCount,AnswerCount,Score,Tags,CreationDate
0,2,1,NaN,what autograding software do folks use for jav...,grading currently either takes me a huge amoun...,9,5402.0,8.0,26,<java><grading><auto-graders>,2017-05-23
1,3,1,NaN,is it possible to ensure division of labor on ...,lets say ive assigned my intro to cs class the...,1,602.0,4.0,10,<group-work>,2017-05-23
2,4,1,NaN,objects lab before inheritance and polymorphism,i am finding it very difficult to create a lab...,3,712.0,11.0,9,<object-oriented-paradigm><labs><ap-computer-s...,2017-05-23
3,6,1,NaN,teaching induction to prove the correctness of...,this is a subject that i have had a lot of tro...,1,413.0,3.0,8,<lesson-ideas><cs-theory><induction>,2017-05-23
4,7,1,NaN,should the first programmingalgorithms class b...,when i was studying my professors had differen...,7,1404.0,9.0,22,<pseudo-code>,2017-05-23


Part way through data cleaning:
1. 'should the first programmingalgorithms ...' - removing punctuation has caused an odd thing here with the / where two words are now joined - check this
2. The Tags column will need to be addressed as well - perhaps the same approach can be applied but I need to make sure that's the best way so that I don't lose anythign when analysing the tags data
3. Removing repeated common words like 'the', 'and' etc

REMINDER - Commit to Github when cleaning is done

In [ ]:
# Setting stop words variable to english setting (NLTK)
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    """
    Removes stopwords like 'The' 'And" etc for cleaner analysis

    Parameters: String of text in both the 'Title' and 'Body' columns containg stopwords

    Returns: Cleaned strings containing no stopwords, keeping the rest of the text
    """
    return ' '.join([word for word in text.split() if word not in stop_words])

qa_df['Title'] = qa_df['Title'].apply(remove_stopwords)
qa_df['Body'] = qa_df['Body'].apply(remove_stopwords)

In [26]:
qa_df.head()

,Id,PostTypeId,ParentId,Title,Body,CommentCount,ViewCount,AnswerCount,Score,Tags,CreationDate
0,2,1,NaN,autograding software folks use java code,grading currently either takes huge amount tim...,9,5402.0,8.0,26,<java><grading><auto-graders>,2017-05-23
1,3,1,NaN,possible ensure division labor group assignment,lets say ive assigned intro cs class first gro...,1,602.0,4.0,10,<group-work>,2017-05-23
2,4,1,NaN,objects lab inheritance polymorphism,finding difficult create lab objects java weve...,3,712.0,11.0,9,<object-oriented-paradigm><labs><ap-computer-s...,2017-05-23
3,6,1,NaN,teaching induction prove correctness algorithms,subject lot trouble clarifying students explai...,1,413.0,3.0,8,<lesson-ideas><cs-theory><induction>,2017-05-23
4,7,1,NaN,first programmingalgorithms class taught pseud...,studying professors different approaches teach...,7,1404.0,9.0,22,<pseudo-code>,2017-05-23


In [ ]:
# Cleaning the tags column - separating the tags into lists instead of the <> 
def clean_tags(tags_string):
    """
    Cleans the 'Tags' Column, changing the <> separators and putting all tags in each column into a list

    Parameters: 'Tags' column containg post subject tags such as '<java>' to be cleaned

    Returns: Cleaned 'Tags' column, where any tags listed are collected and put into one list (per row)
    """
    if pd.isna(tags_string):
        return []
    return tags_string.strip().strip('<>').split('><')

qa_df['Tags'] = qa_df['Tags'].apply(clean_tags)


In [28]:
qa_df.head()

,Id,PostTypeId,ParentId,Title,Body,CommentCount,ViewCount,AnswerCount,Score,Tags,CreationDate
0,2,1,NaN,autograding software folks use java code,grading currently either takes huge amount tim...,9,5402.0,8.0,26,"[java, grading, auto-graders]",2017-05-23
1,3,1,NaN,possible ensure division labor group assignment,lets say ive assigned intro cs class first gro...,1,602.0,4.0,10,[group-work],2017-05-23
2,4,1,NaN,objects lab inheritance polymorphism,finding difficult create lab objects java weve...,3,712.0,11.0,9,"[object-oriented-paradigm, labs, ap-computer-s...",2017-05-23
3,6,1,NaN,teaching induction prove correctness algorithms,subject lot trouble clarifying students explai...,1,413.0,3.0,8,"[lesson-ideas, cs-theory, induction]",2017-05-23
4,7,1,NaN,first programmingalgorithms class taught pseud...,studying professors different approaches teach...,7,1404.0,9.0,22,[pseudo-code],2017-05-23
